In [ ]:
lambdavector = np.linspace (minlambda, maxlambda, num = testpointsdispersion)
wavenumbervector = np.linspace (1e-1, 1e2, num = testpointsdispersion)

j_lmapstability = np.zeros ((testpointsbeta,testpointsj_l_scatter)) #(rows,columns)

#print("first j1map=", j1map)
k = 0
for beta in betavector:
    print ("  >>>> β = ", np.rad2deg(beta), "deg")
    imagmap = np.zeros ((testpointsj_l_scatter, testpointsdispersion)) #(rows,columns)
    mapwp   = [[ ] for i in range (testpointsj_l_scatter)]
    
    j = 0 #for each element of j2vector
    for j_g in j_gscatter: #iterative process for j2        
        i = 0 #for each element of j1vector
        for j_l in j_lscatter: #iterative process for j1
            var4 = var4_0
            var1eq = brenth (equilibrium1, lima, limb)
            
            # parameters used in boundary conditions
            var1_0 = var1eq
            var2_0 = var2 (var1eq)
            var3_0 = var3 (var1eq)
            ref = np.array([var1_0, var2_0, var3_0, var4_0])
            
            # phi=[alpha2, u2, u1, p]ˆT
            Aeval = np.asarray (Amat (var1_0, var2_0, var3_0, var4_0))
            Beval = np.asarray (Bmat (var1_0, var2_0, var3_0, var4_0))
            Ceval = np.asarray (Cmat_lin (var1_0, var2_0, var3_0, var4_0))

            Acomplex = Aeval.dot (1j)
            Bcomplex = Beval.dot (1j)
                        
            #criterion for well-posedness
            h = 0
            for wavenumber in wavenumbervector:
                # wavelength = 2*pi/wavenumber
                m, _ = eig ( (-wavenumber*Bcomplex - Ceval), b =  -Acomplex, overwrite_a = True, overwrite_b = True, check_finite = True) 

                listreal   = []
                listimag   = [] 

                for l in range (len (m)):
                    #print("l =", l)
                    realpart = m [l].real                
                    imagpart = m [l].imag

                    listimag.append (imagpart)
                    listreal.append (realpart)

                if any ([ v < 0 for v in listimag]) :
                    condition      = 'VKH unstable'
                    imagmap [i, h] = min ((np.asarray (listimag)))
                    mapwp[i]       = condition
                else: 
                    condition      = 'VKH stable'
                    imagmap [i, h] = min ( (np.asarray (listimag)))
                    mapwp[i]       = condition
                h +=1
            i += 1 
        j += 1

    # Plot of eigenspectra
    fig, ax = plt.subplots ()
    for jj in range(testpointsj_l_scatter):
        ax.plot (wavenumbervector,
                 -imagmap [jj],
                 liststyles [jj],
                 color = listcolor [0],
                 alpha = alphascatter,
                 label = '$u_{sl}$' + ' = ' + str("%.3f" %j_lscatter [jj]) + ' $\it{m \; s^{-1}}$' + ', ' + '$u_{sg}$' + ' = ' + str("%.3f" %j_gscatter[jj]) + '$\it{m \; s^{-1}}$' + '$, $' \
                     + mapwp[jj] + '.') #+ r'$\alpha$' + ' = ' + str("%.2f" %betavectordeg [k]) + ' , '
        
        ax.set_xscale ('log')
        
        # plt.rcParams ['figure.figsize'] = mapsize
        leg1 = ax.legend (loc = 'best', frameon = True, fontsize = label_size, shadow = True)
        plt.grid (True, which = "both")
        plt.ylim (-1, 2)
        plt.xlim (1e-1, 100)

    
        matplotlib.rc ('xtick', labelsize = label_size)     
        matplotlib.rc ('ytick', labelsize = label_size)
        
        ax.xaxis.set_tick_params(which='major', direction='in', top='on')
        ax.xaxis.set_tick_params(which='minor', direction='in', top='on')
        ax.yaxis.set_tick_params(which='major', direction='in', right='on')
        ax.yaxis.set_tick_params(which='minor', direction='in', right='on')
        
        ax.set_xlabel('$\it{k}$ $[\it{m^{-1}}]$', fontsize = label_size)
        ax.set_ylabel(r'-Im($\omega$) $[\it{s^{-1}}]$', fontsize = label_size)

    plt.axvline(D, 
                inf_omega, 
                sup_omega, 
                label='pyplot vertical line', 
                color = listcolor [0])
    
    # Save plot
    fig.set_size_inches (mapsize)
    plt.savefig('results/figures/diff_maps/fig8_'+ str(k) +'.pdf',
                optimize = True,
                transparent = True,  
                dpi = dpi_elsevier)
    
    plt.show ()
    
    k += 1